In [ ]:
from flask import Flask, render_template, request, jsonify, url_for
import os
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv
from flask_cors import CORS
from functools import lru_cache
import logging
import time
import feedparser   # 👈 RSS ke liye

# -----------------------
# App initialization
# -----------------------
load_dotenv()
app = Flask(__name__)
CORS(app)
logging.basicConfig(level=logging.INFO)

# -----------------------
# Cache-busting for static files
# -----------------------
@app.context_processor
def override_url_for():
    def dated_url_for(endpoint, **values):
        if endpoint == 'static':
            values['q'] = int(time.time())
        return url_for(endpoint, **values)
    return dict(url_for=dated_url_for)

# -----------------------
# API Keys
# -----------------------
API_KEY = os.getenv("NEWS_API_KEY")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

if not API_KEY:
    raise RuntimeError("NEWS_API_KEY nahi mili, .env file me daalein")

analyzer = SentimentIntensityAnalyzer()

# -----------------------
# Helper functions
# -----------------------
def get_sentiment_label(text: str) -> str:
    if not text:
        return "neutral"
    score = analyzer.polarity_scores(text).get("compound", 0)
    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

def ai_summarize(text):
    try:
        import openai
        openai.api_key = OPENAI_KEY
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role":"user", "content": f"Summarize this news in 2-3 sentences:\n{text}"}],
            max_tokens=100
        )
        return response['choices'][0]['message']['content']
    except Exception:
        return text

@lru_cache(maxsize=128)
def fetch_news_from_api(query, language, page):
    url = f"https://newsapi.org/v2/everything?q={requests.utils.quote(query)}&language={language}&page={page}&pageSize=20&apiKey={API_KEY}"
    resp = requests.get(url, timeout=15)
    return resp.json(), resp.status_code

# -----------------------
# Fallback: Google News RSS
# -----------------------
import feedparser   # 👈 upar imports ke sath hi add karo

def fetch_rss_news(query="India", lang="hi"):
    rss_url = f"https://news.google.com/rss/search?q={query}&hl={lang}"
    feed = feedparser.parse(rss_url)
    articles = []
    
    for entry in feed.entries:
        image_url = None
        
        # 1. Kabhi media:content hota hai
        if "media_content" in entry:
            image_url = entry.media_content[0]['url']
        
        # 2. Kabhi links ke andar hota hai
        elif "links" in entry:
            for link in entry.links:
                if link.get("type", "").startswith("image"):
                    image_url = link["href"]
                    break

        # 3. Agar fir bhi na mile to default image
        if not image_url:
            image_url = "/static/default_news.jpg"

        articles.append({
            "title": entry.title,
            "link": entry.link,
            "publishedAt": entry.published if "published" in entry else "",
            "description": entry.summary if "summary" in entry else "",
            "urlToImage": image_url
        })
    return articles

def fetch_news_from_rss(query, language):
    feed_url = f"https://news.google.com/rss/search?q={requests.utils.quote(query)}&hl={language}"
    feed = feedparser.parse(feed_url)
    news_list = []

    for entry in feed.entries[:20]:
        title = entry.get("title")
        desc = entry.get("summary", "")
        summary = ai_summarize(desc)
        sentiment = get_sentiment_label(f"{title}. {summary}".strip())

        news_list.append({
            "title": title,
            "description": desc,
            "summary": summary,
            "url": entry.get("link"),
            "image": None,  # RSS me mostly image nahi hoti
            "source": entry.get("source", {}).get("title", "Google News"),
            "publishedAt": entry.get("published", ""),
            "sentiment": sentiment,
            "category": "General"
        })
    return news_list

# -----------------------
# Routes
# -----------------------
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get_news", methods=["GET"])
def get_news():
    query = request.args.get("query", "technology")
    language = request.args.get("language", "en")
    page = int(request.args.get("page", 1))

    logging.info(f"Fetching news for query: {query}, page: {page}")

    try:
        data, status = fetch_news_from_api(query, language, page)

        if status == 429:
            logging.warning("⚠️ NewsAPI limit reached, switching to Google News RSS")
            news_list = fetch_news_from_rss(query, language)
            return jsonify({"error": False, "items": news_list, "source": "Google News RSS"})

        if status != 200:
            return jsonify({"error": True, "message": data.get("message","Failed to fetch news")}), status

        articles = data.get("articles", [])
        news_list = []
        for a in articles[:20]:
            title = a.get("title")
            desc = a.get("description") or ""
            summary = ai_summarize(desc)
            sentiment = get_sentiment_label(f"{title}. {summary}".strip())

            news_list.append({
                "title": title,
                "description": desc,
                "summary": summary,
                "url": a.get("url"),
                "image": a.get("urlToImage"),
                "source": (a.get("source") or {}).get("name"),
                "publishedAt": a.get("publishedAt"),
                "sentiment": sentiment,
                "category": a.get("source",{}).get("name","General")
            })

        return jsonify({"error": False, "items": news_list, "source": "NewsAPI"})
    except Exception as e:
        logging.error(f"Exception in get_news: {e}")
        # Agar koi aur error ho jaye to bhi RSS backup
        news_list = fetch_news_from_rss(query, language)
        return jsonify({"error": False, "items": news_list, "source": "Google News RSS (fallback)"})

# -----------------------
# Run server
# -----------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.22.249.248:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.22.249.248 - - [18/Aug/2025 15:07:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.22.249.248 - - [18/Aug/2025 15:07:14] "GET /static/style.css?q=1755509834 HTTP/1.1" 200 -
INFO:werkzeug:10.22.249.248 - - [18/Aug/2025 15:07:14] "GET /static/script.js?q=1755509834 HTTP/1.1" 200 -
INFO:root:Fetching news for query: technology, page: 1
INFO:root:Fetching news for query: technology, page: 2
INFO:werkzeug:10.22.249.248 - - [18/Aug/2025 15:07:19] "GET /get_news?query=technology&language=en&page=1 HTTP/1.1" 200 -
INFO:werkzeug:10.22.249.248 - - [18/Aug/2025 15:07:19] "GET /get_news?query=technology&language=en&page=2 HTTP/1.1" 200 -
INFO:root:Fetching news for query: entertainment, page: 1
INFO:root:Fetc

In [6]:
pip install vaderSentiment


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install feedparser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=5ca863dad143327319fa59b48362d2e65ed5a366ce9187c242fcdcde0044ff7d
  Stored in directory: c:\users\nr143\appdata\local\pip\cache\wheels\3d\4d\ef\37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

   -------------------- ------------------- 1/2 [feedparser]
   ---------------------------------------- 2/2 [feedparser]

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sgmllib3k'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [33]:
pip install flask-cors


Note: you may need to restart the kernel to use updated packages.


In [34]:
!pip install flask-cors

In [7]:
pip show vaderSentiment

Name: vaderSentiment
Version: 3.3.2
Summary: VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.
Home-page: https://github.com/cjhutto/vaderSentiment
Author: C.J. Hutto
Author-email: cjhutto@gatech.edu
License: MIT License
Location: c:\Users\nr143\anaconda3\Lib\site-packages
Requires: requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.
